Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from Brillar Bank

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from helpers.test_data import get_urls

loader = WebBaseLoader(get_urls())

data = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


Split the Loaded Data into Chunks

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
texts = text_splitter.split_documents(data)

Get Embedding Model

In [4]:
from langchain_openai import OpenAIEmbeddings

embedding = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=256)

Instantiate Elasticsearch Store

In [5]:
from langchain_elasticsearch import ElasticsearchStore, SparseVectorStrategy, BM25Strategy
import os

es_cloud_id = os.getenv("ELASTIC_CLOUD_ID")
es_api_key = os.getenv("ELASTIC_API_KEY")

bm25_store = ElasticsearchStore.from_documents(
  documents=texts,
  es_cloud_id=es_cloud_id,
  es_api_key=es_api_key,
  index_name="bm25_sparse",
  strategy=BM25Strategy()
)

sparse_vector_store = ElasticsearchStore.from_documents(
    documents=texts,
    es_cloud_id=es_cloud_id,
    es_api_key=es_api_key,
    index_name="sparse_vector_bm25",
    strategy=SparseVectorStrategy(model_id=".elser_model_2"),
    bulk_kwargs={
      "request_timeout": 60
    }
)

Get LLM

In [6]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

Get Retriever

In [7]:
from langchain.retrievers import EnsembleRetriever

bm25_retriever = bm25_store.as_retriever(search_kwargs={"k": 5})
sparse_retriever = sparse_vector_store.as_retriever(search_kwargs={"k": 5})

retriever = EnsembleRetriever(retrievers=[bm25_retriever, sparse_retriever])

Create Chain

In [8]:
from helpers.conversation_retrieval_chain import create_conversational_retrieval_chain, invoke_conversational_retrieval_chain

chain = create_conversational_retrieval_chain(llm, retriever)

Invoke Chain

In [9]:
question = "what are the interest rates for fixed deposit?"
result = invoke_conversational_retrieval_chain(chain, question)
result

{'input': 'what are the interest rates for fixed deposit?',
 'chat_history': [],
 'context': [Document(metadata={'source': 'https://win066.wixsite.com/brillar-bank/flexi-fixed-deposit', 'title': 'Flexi Fixed Deposit | Brillar Bank', 'language': 'en'}, page_content='Flexi Fixed Deposit | Brillar Bank\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\ntop of pageThis site was designed with the .com website builder. Create your website today.Start NowHomeDepositsMoreUse tab to navigate through the menu items.Available for Registered Users\xa0 \xa0 \xa0 \xa0Learn More\xa0\n\n\n\n\nFlexi Fixed Deposit @Brillar BankA truly flexible fixed deposit accountBrillar Flexi Fixed Deposit rewards you with attractive interest and auto-sweep facility from Flexi Fixed Deposit to Flexi One Account or vice versa.Get StartedLearn More\xa0\n\n\n\n\nFeatures and BenefitsEarn attractive FD interest for 12-month tenureWhen your Flexi One Account exceeds RM30,000, it will be transferred to your Flexi FD Account to earn attra